In [1]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential,layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D,Softmax
from tensorflow.keras.layers import Conv3D,LayerNormalization,ReLU,MultiHeadAttention,GlobalAveragePooling3D
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
import einops

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d asefjamilajwad/car-crash-dataset-ccd

Dataset URL: https://www.kaggle.com/datasets/asefjamilajwad/car-crash-dataset-ccd
License(s): other
100% 7.61G/7.61G [01:44<00:00, 120MB/s]
100% 7.61G/7.61G [01:44<00:00, 78.0MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/car-crash-dataset-ccd.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

##Data Preprocessing

In [ ]:
df1=pd.read_csv("/content/Severity.csv")
# num_videos=df1.shape[0]
num_videos=5

In [ ]:
from google.colab import files
import shutil
# Define source and destination paths
source_path = "/content/CrashBest/"
dest_path = "/content/test1/"
os.makedirs(dest_path, exist_ok=True)

# List all files in the source directory
all_files = os.listdir(source_path)

# Copy the first 100 files
for video_name in range(1,num_videos+1):
  for i in range(1,51):
    frame_number = str(i).zfill(2)  # Ensure two digits with leading zeros
    file_name = f"C_{video_name:06}_{frame_number}.jpg"

    shutil.copy(source_path + file_name, dest_path + file_name)



In [ ]:
BASE_PATH =dest_path
TABLENAME = "/content/Crash_Table.csv"
classify_model_path="/content"

In [ ]:
df=pd.read_csv(TABLENAME)
col_class="Severity of the Crash"
df[col_class] = df1[col_class]
df[col_class] = df[col_class].fillna('minor')#fill NAN with minor of col_class
df[col_class] = df[col_class].str.lower()#convert into lower case
num_classes = df[col_class].unique()
label_encoding = {category: [1 if category == unique else 0 for unique in num_classes] for category in num_classes}
img_size=[225,225]

In [ ]:
df.head()

,vidname,frame_1,frame_2,frame_3,frame_4,frame_5,frame_6,frame_7,frame_8,frame_9,...,frame_47,frame_48,frame_49,frame_50,startframe,youtubeID,timing,weather,egoinvolve,Severity of the Crash
0,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,285,0,Day,Normal,Yes,moderate
1,2,0,0,0,0,0,0,0,0,0,...,1,1,1,1,646,0,Day,Normal,Yes,severe
2,3,0,0,0,0,0,0,0,0,0,...,1,1,1,1,972,0,Day,Normal,Yes,minor
3,4,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1257,0,Day,Normal,Yes,moderate
4,5,0,0,0,0,0,0,0,0,0,...,1,1,1,1,462,10,Day,Snowy,Yes,moderate


In [ ]:
def load_video(video_name):
  frames = []
  for i in range(1, 51):
    frame_number = str(i).zfill(2)  # Ensure two digits with leading zeros
    file_name = f"C_{video_name:06}_{frame_number}.jpg"  # Six-digit video name
    frame_path = os.path.join(BASE_PATH, file_name)
    if os.path.exists(frame_path):
      img = cv2.imread(frame_path)  # Use OpenCV to read image
      img = cv2.resize(img, (img_size[0], img_size[1]))  # Resize image
      frames.append(img)
  return tf.convert_to_tensor(frames, dtype=tf.float32)

In [ ]:
def attention_func(data):
  accident_frame_atten=0.8
  non_accident_frame_atten=0.2
  frame_scene=[]
  for index in range(1,51):
    frame_num=f"frame_{index}"
    if df[frame_num][data]==1:
      frame_scene.append(accident_frame_atten)
    else:
      frame_scene.append(non_accident_frame_atten)
  return tf.convert_to_tensor(frame_scene, dtype=tf.float32)


In [ ]:
from sklearn.model_selection import train_test_split
def Create_dataset():
  images_list=[]
  labels_list=[]
  attention_list=[]
  for data in range(0,num_videos):
    frame=load_video(df['vidname'][data])
    images_list.append(frame)
    labels_list.append(label_encoding[df[col_class][data]])
    attention_list.append(attention_func(data))
  image_array=np.array(images_list)
  labels_array=np.array(labels_list)
  attention_array=np.array(attention_list)
  train_images, temp_images, train_labels, temp_labels, train_attentions, temp_attentions = train_test_split(
      image_array, labels_array, attention_array, train_size=0.7, random_state=42, shuffle=True
  )

  val_images, test_images, val_labels, test_labels, val_attentions, test_attentions = train_test_split(
      temp_images, temp_labels, temp_attentions, test_size=0.5, random_state=42, shuffle=True
  )
  train_images=tf.convert_to_tensor(train_images)
  val_images=tf.convert_to_tensor(val_images)
  test_images=tf.convert_to_tensor(test_images)
  train_attentions=tf.convert_to_tensor(train_attentions)
  val_attentions=tf.convert_to_tensor(val_attentions)
  test_attentions=tf.convert_to_tensor(test_attentions)
  train_labels=tf.convert_to_tensor(train_labels)
  val_labels=tf.convert_to_tensor(val_labels)
  test_labels=tf.convert_to_tensor(test_labels)
  return train_images, val_images, test_images,  train_labels,val_labels, test_labels,  train_attentions, val_attentions, test_attentions
train_images, val_images, test_images,  train_labels,val_labels, test_labels,  train_attentions, val_attentions, test_attentions=Create_dataset()


In [ ]:
train_images.shape

TensorShape([3, 50, 225, 225, 3])

##Model

In [11]:

class Conv2Plus1D(tf.keras.Model):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = Sequential([
            # Spatial decomposition
            Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            # Temporal decomposition
            Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, inputs):
        return self.seq(inputs)

class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            LayerNormalization(),
            ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos


class video_analyze(tf.keras.Model):
    def __init__(self, HEIGHT, WIDTH):
        super().__init__()
        self.seq1 = Sequential([
            Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same'),
            BatchNormalization(),
            ReLU(),
            ResizeVideo(HEIGHT // 2, WIDTH // 2)
        ])
        self.seq2=Sequential([
            ResidualMain(16,(3,3,3)),
            ResizeVideo(HEIGHT // 4, WIDTH // 4),
            ResidualMain(32,(3,3,3)),
            ResizeVideo(HEIGHT // 8, WIDTH // 8),
            ResidualMain(64,(3,3,3)),
            ResizeVideo(HEIGHT // 16, WIDTH // 16),
            ResidualMain(128,(3,3,3)),
        ])
        self.seq3 = Sequential([
            GlobalAveragePooling3D(),
            Flatten(),
            Dense(512, activation='relu')
        ])

    def call(self, video):
        x = self.seq1(video)
        x=self.seq2(x)
        x = self.seq3(x)
        return x



In [6]:
class CombineTemporalLayers(tf.keras.Model):
    def __init__(self):
        super(CombineTemporalLayers, self).__init__()
        self.conv3d = tf.keras.layers.Conv3D(
            filters=3,
            kernel_size=(50, 1, 1),
            strides=(1, 1, 1),
            padding='valid',
            activation=None,
            use_bias=False
        )
        self.self_attention = MultiHeadAttention(num_heads=4, key_dim=3)

    def call(self, inputs):
        # Apply Conv3D to combine the layers
        x = self.conv3d(inputs)
        # Squeeze the combined layer dimension
        x = tf.squeeze(x, axis=1)

        return x
class Encoder(tf.keras.Model):
  def __init__(self, filters, kernel_size):
    super(Encoder,self).__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        LayerNormalization(),
        ReLU()
    ])
    self.combine=CombineTemporalLayers()
    self.self_attention = MultiHeadAttention(num_heads=4, key_dim=filters)
  def call(self, x):
   x = self.seq(x)
        # Get the shape dynamically
   batch_size = tf.shape(x)[0]
   seq_length = tf.shape(x)[1]
   height = tf.shape(x)[2]
   width = tf.shape(x)[3]
   channels = tf.shape(x)[4]
   x = tf.reshape(x, tf.stack([batch_size, seq_length, height * width * channels]))
   x = self.self_attention(x, x)

   x = tf.reshape(x, tf.stack([batch_size, seq_length, height, width, channels]))
   return self.combine(x)


In [7]:
class inception(keras.Model):
    def __init__(self,num_first_filter,num_second_filter):
        super().__init__()
        self.conv1=Conv2D(num_first_filter,kernel_size=(1,1),strides=1,padding='valid',activation='relu')
        self.bn1=BatchNormalization()
        self.conv2A=Conv2D(num_second_filter,kernel_size=(1,1),strides=1,padding='valid',activation='relu')
        self.bn2A=BatchNormalization()
        self.conv2B=Conv2D(num_second_filter,kernel_size=(3,3),strides=1,padding='same',activation='relu')
        self.bn2B=BatchNormalization()
        self.conv2C=Conv2D(num_second_filter,kernel_size=(3,3),strides=1,padding='same',activation='relu')
        self.bn2C=BatchNormalization()
        self.pool=MaxPooling2D(pool_size=2, strides=1, padding='same')
    def call(self,inputs):
        x=self.bn2B(self.conv2B(self.bn1(self.conv1(inputs))))
        # print(x.shape)
        x1=self.bn2C(self.conv2C(x))
        # print(x1.shape)
        x2=self.bn2A(self.conv2A(self.pool(inputs)))
        x3=self.bn2A(self.conv2A(inputs))

        output=tf.concat([x,x1,x2,x3],-1)
        return output

class inception_reduction(keras.Model):
    def __init__(self,num_first_filter,num_second_filter,num_third_filter):
        super(inception_reduction,self).__init__()
        self.conv1=Sequential([Conv2D(num_first_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv1A=Sequential([Conv2D(num_first_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2A=Sequential([Conv2D(num_second_filter,kernel_size=(3,3),strides=2,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2B=Sequential([Conv2D(num_second_filter,kernel_size=(3,3),strides=2,padding='same'),
                                BatchNormalization(),
                                ReLU()])

        self.pool=MaxPooling2D(pool_size=2, strides=2)
        self.conv3=Sequential([Conv2D(num_third_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])

    def call(self,inputs):
        # x=self.conv64(inputs)
        x=self.conv1A(self.conv1(inputs))
        # print(x.shape)
        x=self.conv2A(x)
        # print(x.shape)
        x1=self.conv2B(self.conv1(inputs))
        x2=self.conv3(self.pool(inputs))
        output=tf.concat([x,x1,x2],-1)
        return output
class inception_resnet(keras.Model):
    def __init__(self,num_first_filter,num_second_filter):
        super(inception_resnet,self).__init__()
        self.conv1=Sequential([Conv2D(num_first_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2=Sequential([Conv2D(num_second_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv1A=Sequential([Conv2D(num_first_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2A=Sequential([Conv2D(num_second_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2B=Sequential([Conv2D(num_second_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])

    def call(self,inputs):
        x=self.conv1A(self.conv1(inputs))
        # print(x.shape)
        x=self.conv2A(x)
        # print(x.shape)
        x1=self.conv2B(self.conv1(inputs))
        # print(x1.shape)
        x2=self.conv2(inputs)
        # print(x2.shape)
        output=tf.concat([x,x1,x2],-1)

        return output
class inception_resnet_V2(keras.Model):
    def __init__(self,num_first_filter,num_second_filter,num_third_filter):
        super(inception_resnet_V2,self).__init__()
        self.conv1=Sequential([Conv2D(num_first_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv3=Sequential([Conv2D(num_third_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2=Sequential([Conv2D(num_second_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv3A=Sequential([Conv2D(num_third_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.conv3B=Sequential([Conv2D(num_third_filter,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])

        self.pool=MaxPooling2D(pool_size=2, strides=1,padding='same')
        self.conv3C=Sequential([Conv2D(num_third_filter,kernel_size=(1,1),strides=1,padding='valid'),
                                BatchNormalization(),
                                ReLU()])

    def call(self,inputs):
        x=self.conv2(self.conv1(inputs))
        # print(x.shape)
        x=self.conv3A(x)
        # print(x.shape)
        x1=self.conv3B(self.conv1(inputs))
        # print(x1.shape)
        x2=self.conv3(inputs)
        # print(x2.shape)
        x3=self.conv3C(self.pool(inputs))
        output=tf.concat([x,x1,x2,x3],-1)
        return output
class Attention(keras.Model):
    def __init__(self):
        super(Attention,self).__init__()
        self.conv1=Conv2D(1024,kernel_size=(1,1),strides=1,padding='valid',activation='relu')
        self.bn1=BatchNormalization()
        self.softmax=Softmax(axis=-1)
    def call(self,input1,input2):
        f=self.bn1(self.conv1(input1))
        G=tf.multiply(f,input2)
        p=self.softmax(G)
        temp=tf.multiply(p,f)
        attn=tf.reduce_sum(temp,axis=[1,2])
        return attn
class Neural_Network(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1=Sequential([Conv2D(32,kernel_size=(3,3),strides=2,padding='valid'),
                                BatchNormalization(),
                                ReLU()])
        self.conv2=Sequential([Conv2D(64,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.pool3=MaxPooling2D(pool_size=2, strides=2, padding='valid')
        self.inception4=inception(48,64)
        self.inception5=inception(48,64)
        self.inception_red6=inception_reduction(64,96,192)
        self.inception_red10=inception_reduction(96,112,384)
        self.inception_resnet7=inception_resnet(96,128)
        self.inception_resnet8=inception_resnet(96,128)
        self.inception_resnet11=inception_resnet_V2(112,128,152)
        self.inception_resnet12=inception_resnet_V2(112,128,152)
        self.inception_red14=inception_reduction(112,128,608)
        self.conv15=Sequential([Conv2D(1024,kernel_size=(3,3),strides=1,padding='same'),
                                BatchNormalization(),
                                ReLU()])
        self.pool16=AveragePooling2D(pool_size=(7,7))
        self.attention1=Attention()
        self.attention2=Attention()
        self.ffn18=Dense(512, activation='relu')
        self.bn18=BatchNormalization()
        # self.ffn19=Dense(2, activation='relu')
        # self.softmax19=Softmax()
    def call(self,inputs):
        x_1=self.conv1(inputs)
        # print(x_1.shape)
        x_2=self.conv2(x_1)
        # print(x_2.shape)
        x_3=self.pool3(x_2)
        # print(x_3.shape)
        x_4=self.inception4(x_3)
        x_5=self.inception5(x_4)
        x_6=self.inception_red6(x_5)
        # print(x_6.shape)
        x_7=self.inception_resnet7(x_6)
        # print(x_7.shape)
        x_8=self.inception_resnet8(x_7)
        x_10=self.inception_red10(x_8)
        x_11=self.inception_resnet11(x_10)
        x_12=self.inception_resnet12(x_11)
        x_14=self.inception_red14(x_12)
        x_15=self.conv15(x_14)
        x_16=self.pool16(x_15)
        x_9=self.attention1(x_8,x_16)
        x_13=self.attention2(x_12,x_16)
        x_17=tf.concat([x_9,x_13],-1)
        x_18=self.bn18(self.ffn18(x_17))
        # x_19=self.ffn19(x_18)
        # logits_softmax, logits_centers = tf.split(x_19, [2, 2], axis=-1)
        # print(logits_centers.shape)
        # output=self.softmax19(x_19)
        output=x_18
        # print(f'output{output.shape}')
        return output


In [9]:
class Decision_Network(keras.Model):
    def __init__(self,num_classes):
        super(Decision_Network,self).__init__()
        self.Encoder = Encoder(filters=3, kernel_size=(3, 3, 3))
        self.NN1 = Neural_Network()
        self.video_cls=video_analyze(225,225)
        self.seq1=Sequential([
            BatchNormalization(),
            Dropout(0.2),
            Dense(256,activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(128, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(32, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(4, activation='softmax')

        ])
    def call(self, inputs):
        x = self.Encoder(inputs)
        # print(x.shape)
        x=self.NN1(x)+self.video_cls(inputs)
        # print(x.shape)

        return self.seq1(x)

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam", metrics=["accuracy"])

In [ ]:
history = model.fit(train_images, train_labels, epochs=1, validation_data=(val_images, val_labels))


(None, 225, 225, 3)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 28, 28, 384)
(None, 28, 28, 384)
output(None, 512)
(None, 512)
(None, 225, 225, 3)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 28, 28, 384)
(None, 28, 28, 384)
output(None, 512)
(None, 512)
1/1 [==============================] - ETA: 0s - loss: -1.2085 - accuracy: 0.0000e+00(None, 225, 225, 3)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 28, 28, 384)
(None, 28, 28, 384)
output(None, 512)
(None, 512)
1/1 [==============================] - 51s 51s/step - loss: -1.2085 - accuracy: 0.0000e+00 - val_loss: 0.8227 - val_accuracy: 0.0000e+00


In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate([test_images,test_attentions], test_labels)
print("Test accuracy:", test_acc)

(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 28, 28, 384)
(None, 28, 28, 384)
output(None, 512)
1/1 [==============================] - 2s 2s/step - loss: 0.6920 - accuracy: 1.0000
Test accuracy: 1.0
